In [1]:
import sys, os, time, shutil, random
from pathlib import Path
_cwd = os.getcwd()
os.chdir(Path(_cwd)/'..')
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))
# %load_ext autoreload
# %autoreload 2
# %pdb
from itertools import product, cycle
from heapq import heapify, heappush, heappop
import h5py as hf
import cv2
import omnifig as fig
import numpy as np
np.set_printoptions(linewidth=120)
import pickle
from tabulate import tabulate
from tqdm.notebook import tqdm
import torch
from torch import nn
import torch.nn.functional as F
import torch.distributions as distrib
import torch.multiprocessing as mp
# import torchvision.models
from torch.utils.data import Dataset, DataLoader, TensorDataset
import networkx as nx

%matplotlib notebook
import matplotlib.pyplot as plt
#plt.switch_backend('Qt5Agg') #('Qt5Agg')
import matplotlib.patches as mpatches
import matplotlib.pyplot as plt
import matplotlib.cm
from matplotlib import animation
import matplotlib as mpl
from matplotlib.collections import PatchCollection
# mpl.rc('image', cmap='gray')
import seaborn as sns
import pandas as pd
from sklearn.preprocessing import normalize

fig.initialize()

import omnilearn as learn
from omnilearn import models
from omnilearn import util
from omnilearn.data import InterventionSamplerBase
from omnilearn import viz as viz_util

from sklearn.cluster import MeanShift, estimate_bandwidth, DBSCAN, KMeans
from sklearn.datasets import make_blobs
from sklearn.decomposition import PCA

from src import sample_full_interventions, response_mat, conditioned_reponses
from src.responses import sample_full_interventions

# import gpumap
from c3linearize import linearize, class_graph

fig.get_config().silence();

In [5]:
rnames = '''3ds_vae1_d-conv_s1_210523-164722	3ds_vae1_d-conv_d24-s1_210523-164723
3ds_vae1_d-conv_s2_210523-164723	3ds_vae1_d-conv_d24-s2_210523-164722
3ds_vae2_d-conv_s1_210523-164723	3ds_vae2_d-conv_d24-s1_210523-164722
3ds_vae2_d-conv_s2_210523-164722	3ds_vae2_d-conv_d24-s2_210523-164722
3ds_vae4_d-conv_s1_210523-164723	3ds_vae4_d-conv_d24-s1_210523-164722
3ds_vae4_d-conv_s2_210523-164722	3ds_vae4_d-conv_d24-s2_210523-164722
3ds_vae8_d-conv_s1_210523-164722	3ds_vae8_d-conv_d24-s1_210523-164723
3ds_vae8_d-conv_s2_210523-164722	3ds_vae8_d-conv_d24-s2_210523-164722'''.replace('\t', '\n').split('\n')

rnames = '''toy_vae1_e-c12_d-c12_d6-ch128-strd-trans_210522-020221
toy_vae1_e-c12_d-c12_d8-ch128-strd-trans_210522-020221
toy_vae1_e-c12_d-c12_d10-ch128-strd-trans_210522-020222
toy_vae1_e-c12_d-c12_ch128-strd-trans_210522-015012
toy_vae1_e-c12_d-c12_d16-ch128-strd-trans_210522-020223
toy_vae1_e-c12_d-c12_d32-ch128-strd-trans_210522-020222'''.split('\n')

rnames = '''real_vae1_e-c12_d-c12_d6-ch128-strd-trans_210522-020221
real_vae1_e-c12_d-c12_d8-ch128-strd-trans_210522-020222
real_vae1_e-c12_d-c12_d10-ch128-strd-trans_210522-020221
real_vae1_e-c12_d-c12_d12-ch128-strd-trans_210522-020222
real_vae1_e-c12_d-c12_d16-ch128-strd-trans_210522-020222
real_vae1_e-c12_d-c12_d32-ch128-strd-trans_210522-020221'''.split('\n')

In [6]:
tmpl = 'fleeb@$STOKKUR:/is/ei/fleeb/workspace/chome/trained_nets/{}'
cmd = 'rsync -rvz {} .'

In [7]:
print(cmd.format(' '.join(tmpl.format(n) for n in rnames)))

rsync -rvz fleeb@$STOKKUR:/is/ei/fleeb/workspace/chome/trained_nets/real_vae1_e-c12_d-c12_d6-ch128-strd-trans_210522-020221 fleeb@$STOKKUR:/is/ei/fleeb/workspace/chome/trained_nets/real_vae1_e-c12_d-c12_d8-ch128-strd-trans_210522-020222 fleeb@$STOKKUR:/is/ei/fleeb/workspace/chome/trained_nets/real_vae1_e-c12_d-c12_d10-ch128-strd-trans_210522-020221 fleeb@$STOKKUR:/is/ei/fleeb/workspace/chome/trained_nets/real_vae1_e-c12_d-c12_d12-ch128-strd-trans_210522-020222 fleeb@$STOKKUR:/is/ei/fleeb/workspace/chome/trained_nets/real_vae1_e-c12_d-c12_d16-ch128-strd-trans_210522-020222 fleeb@$STOKKUR:/is/ei/fleeb/workspace/chome/trained_nets/real_vae1_e-c12_d-c12_d32-ch128-strd-trans_210522-020221 .


In [4]:
print('\n'.join(tmpl.format(n) for n in rnames))

rsync -rvz fleeb@$STOKKUR:/is/ei/fleeb/workspace/chome/trained_nets/3ds_vae1_d-conv_s1_210523-164722 .
rsync -rvz fleeb@$STOKKUR:/is/ei/fleeb/workspace/chome/trained_nets/3ds_vae1_d-conv_d24-s1_210523-164723 .
rsync -rvz fleeb@$STOKKUR:/is/ei/fleeb/workspace/chome/trained_nets/3ds_vae1_d-conv_s2_210523-164723 .
rsync -rvz fleeb@$STOKKUR:/is/ei/fleeb/workspace/chome/trained_nets/3ds_vae1_d-conv_d24-s2_210523-164722 .
rsync -rvz fleeb@$STOKKUR:/is/ei/fleeb/workspace/chome/trained_nets/3ds_vae2_d-conv_s1_210523-164723 .
rsync -rvz fleeb@$STOKKUR:/is/ei/fleeb/workspace/chome/trained_nets/3ds_vae2_d-conv_d24-s1_210523-164722 .
rsync -rvz fleeb@$STOKKUR:/is/ei/fleeb/workspace/chome/trained_nets/3ds_vae2_d-conv_s2_210523-164722 .
rsync -rvz fleeb@$STOKKUR:/is/ei/fleeb/workspace/chome/trained_nets/3ds_vae2_d-conv_d24-s2_210523-164722 .
rsync -rvz fleeb@$STOKKUR:/is/ei/fleeb/workspace/chome/trained_nets/3ds_vae4_d-conv_s1_210523-164723 .
rsync -rvz fleeb@$STOKKUR:/is/ei/fleeb/workspace/chome/tr

In [11]:
tmpl = 'unbuffer fig eval-multiple-metrics eval/mpi --cat real --run-name {}'
# tmpl = 'unbuffer fig eval-multiple-metrics eval/3ds --run-name {}'

In [12]:
print('\n'.join(tmpl.format(n) for n in rnames))

unbuffer fig eval-multiple-metrics eval/mpi --cat real --run-name real_vae1_e-c12_d-c12_d6-ch128-strd-trans_210522-020221
unbuffer fig eval-multiple-metrics eval/mpi --cat real --run-name real_vae1_e-c12_d-c12_d8-ch128-strd-trans_210522-020222
unbuffer fig eval-multiple-metrics eval/mpi --cat real --run-name real_vae1_e-c12_d-c12_d10-ch128-strd-trans_210522-020221
unbuffer fig eval-multiple-metrics eval/mpi --cat real --run-name real_vae1_e-c12_d-c12_d12-ch128-strd-trans_210522-020222
unbuffer fig eval-multiple-metrics eval/mpi --cat real --run-name real_vae1_e-c12_d-c12_d16-ch128-strd-trans_210522-020222
unbuffer fig eval-multiple-metrics eval/mpi --cat real --run-name real_vae1_e-c12_d-c12_d32-ch128-strd-trans_210522-020221
